In [1]:
import os
import pandas as pd
import numpy as np
import json
import pprint
import re
import nltk
import csv

In [2]:
data_dir = "data"
data_folders = os.listdir(data_dir)

for data_folder in data_folders:
    print(data_folder,'\n',os.listdir(os.path.join(data_dir,data_folder)))
    print('-'*30)

empatheticdialogues 
 ['test.csv', 'train.csv', 'valid.csv']
------------------------------
goemotions 
 ['goemotions_1.csv', 'goemotions_2.csv', 'goemotions_3.csv']
------------------------------
processed 
 ['all_ed_data.csv', 'ed_for_goemotions.tsv', 'ed_for_goemotions.tsv.eval_results.txt', 'ed_for_goemotions.tsv.label_predictions.tsv', 'ed_for_goemotions.tsv.predictions.tsv', 'ed_for_goemotions.tsv.tf_record', 'all_dd_data.tsv', 'all_meld_data.tsv', 'all_names.csv', 'dd_for_goemotions.tsv', 'meld_for_goemotions.tsv', 'dd_for_goemotions.tsv.eval_results.txt', 'dd_for_goemotions.tsv.label_predictions.tsv', 'dd_for_goemotions.tsv.predictions.tsv', 'dd_for_goemotions.tsv.tf_record', 'meld_for_goemotions.tsv.eval_results.txt', 'meld_for_goemotions.tsv.label_predictions.tsv', 'meld_for_goemotions.tsv.predictions.tsv', 'meld_for_goemotions.tsv.tf_record']
------------------------------
dailydialogues 
 ['dialogues_act.txt', 'dialogues_emotion.txt', 'dialogues_text.txt', 'dialogues_topic.

### ED Results

In [3]:
data_folder='processed'
file_name='ed_for_goemotions.tsv.label_predictions.tsv'
ed_labels = pd.read_csv(os.path.join(data_dir, data_folder, file_name), sep='\t')
print(ed_labels.shape)
ed_labels.head()

(12078, 7)


,text,emotion_1,prob_1,emotion_2,prob_2,emotion_3,prob_3
0,"Today, as i was leaving for work in the mornin...",neutral,0.3973,annoyance,0.1223,disapproval,0.1175
1,Are you fine now?,curiosity,0.4739,neutral,0.2788,confusion,0.1479
2,"Yeah, i'm doing alright now, but with minor in...",approval,0.2368,admiration,0.1763,neutral,0.1256
3,Cool :) Is your car damaged a lot?,curiosity,0.5304,neutral,0.1974,confusion,0.1616
4,"The car was badly damaged, i veered outside th...",neutral,0.3217,annoyance,0.1912,disapproval,0.1674


In [4]:
file_name='ed_for_goemotions.tsv'
ed_data = pd.read_csv(os.path.join(data_dir, data_folder, file_name), sep='\t', header=None, quoting=csv.QUOTE_NONE)
ed_data.columns = ["utterance", "label", "id"]
print(ed_data.shape)
ed_data.head()

(12078, 3)


,utterance,label,id
0,"Today, as i was leaving for work in the mornin...",0.0,hit:3_conv:6_1_6
1,Are you fine now?,0.0,hit:3_conv:6_2_7
2,"Yeah, i'm doing alright now, but with minor in...",0.0,hit:3_conv:6_3_6
3,Cool :) Is your car damaged a lot?,0.0,hit:3_conv:6_4_7
4,"The car was badly damaged, i veered outside th...",0.0,hit:3_conv:6_5_6


In [5]:
ed_data.label = ed_labels.emotion_1
ed_data.head()

,utterance,label,id
0,"Today, as i was leaving for work in the mornin...",neutral,hit:3_conv:6_1_6
1,Are you fine now?,curiosity,hit:3_conv:6_2_7
2,"Yeah, i'm doing alright now, but with minor in...",approval,hit:3_conv:6_3_6
3,Cool :) Is your car damaged a lot?,curiosity,hit:3_conv:6_4_7
4,"The car was badly damaged, i veered outside th...",neutral,hit:3_conv:6_5_6


In [32]:
ed_idx = ed_data.id

convo_id = list()
turn_id = list()

for x in ed_idx:
    y = re.split(":", str(x))
    print(y[2])

6_1_6
6_2_7
6_3_6
6_4_7
6_5_6
9_1_8
9_2_4
9_3_8
9_4_4
12_1_6
12_2_11
12_3_6
12_4_11
17_1_8
17_2_4
17_3_8
17_4_4
18_1_12
18_2_10
18_3_12
18_4_10
20_1_13
20_2_14
20_3_13
20_4_14
38_1_26
38_2_18
38_3_26
38_4_18
62_1_44
62_2_11
62_3_44
62_4_11
62_5_44
68_1_12
68_2_45
68_3_12
68_4_45
77_1_12
77_2_11
77_3_12
77_4_11
102_1_51
102_2_11
102_3_51
102_4_11
102_5_51
110_1_51
110_2_1
110_3_51
110_4_1
110_5_51
217_1_51
217_2_27
217_3_51
217_4_27
226_1_51
226_2_4
226_3_51
226_4_4
239_1_51
239_2_55
239_3_51
239_4_55
243_1_51
243_2_4
243_3_51
243_4_4
247_1_51
247_2_11
247_3_51
247_4_11
251_1_51
251_2_4
251_3_51
251_4_4
254_1_51
254_2_4
254_3_51
254_4_4
256_1_51
256_2_4
256_3_51
256_4_4
262_1_51
262_2_4
262_3_51
262_4_4
281_1_13
281_2_4
281_3_13
281_4_4
292_1_60
292_2_61
292_3_60
292_4_61
292_5_60
292_6_61
293_1_61
293_2_60
293_3_61
293_4_60
293_5_61
293_6_60
317_1_51
317_2_10
317_3_51
317_4_10
319_1_60
319_2_10
319_3_60
319_4_10
319_5_60
323_1_51
323_2_35
323_3_51
323_4_35
324_1_61
324_2_63
324_3_61
32

1353_1_183
1353_2_205
1353_3_183
1353_4_205
1356_1_189
1356_2_202
1356_3_189
1356_4_202
1361_1_183
1361_2_70
1361_3_183
1361_4_70
1371_1_189
1371_2_181
1371_3_189
1371_4_181
1376_1_183
1376_2_188
1376_3_183
1376_4_188
1380_1_189
1380_2_187
1380_3_189
1380_4_187
1389_1_189
1389_2_203
1389_3_189
1389_4_203
1401_1_208
1401_2_203
1401_3_208
1401_4_203
1401_5_208
1408_1_189
1408_2_188
1408_3_189
1408_4_188
1411_1_208
1411_2_202
1411_3_208
1411_4_202
1411_5_208
1413_1_183
1413_2_34
1413_3_183
1413_4_34
1421_1_183
1421_2_34
1421_3_183
1421_4_34
1429_1_183
1429_2_34
1429_3_183
1429_4_34
1438_1_189
1438_2_188
1438_3_189
1438_4_188
1453_1_189
1453_2_210
1453_3_189
1453_4_210
1455_1_101
1455_2_187
1455_3_101
1455_4_187
1458_1_153
1458_2_209
1458_3_153
1458_4_209
1464_1_189
1464_2_210
1464_3_189
1464_4_210
1475_1_218
1475_2_215
1475_3_218
1475_4_215
1481_1_101
1481_2_210
1481_3_101
1481_4_210
1484_1_218
1484_2_126
1484_3_218
1484_4_126
1489_1_61
1489_2_11
1489_3_61
1489_4_11
1494_1_61
1494_2_220
1

4571_2_262
4571_3_250
4571_4_262
4593_1_250
4593_2_1
4593_3_250
4593_4_1
4598_1_259
4598_2_1
4598_3_259
4598_4_1
4610_1_111
4610_2_1
4610_3_111
4610_4_1
4637_1_111
4637_2_10
4637_3_111
4637_4_10
4642_1_60
4642_2_10
4642_3_60
4642_4_10
4642_5_60
4648_1_44
4648_2_7
4648_3_44
4648_4_7
4702_1_44
4702_2_20
4702_3_44
4702_4_20
4707_1_60
4707_2_10
4707_3_60
4707_4_10
4707_5_60
4708_1_111
4708_2_259
4708_3_111
4708_4_259
4709_1_259
4709_2_111
4709_3_259
4709_4_111
4710_1_44
4710_2_277
4710_3_44
4710_4_277
4719_1_259
4719_2_231
4719_3_259
4719_4_231
4723_1_259
4723_2_228
4723_3_259
4723_4_228
4726_1_259
4726_2_277
4726_3_259
4726_4_277
4728_1_259
4728_2_7
4728_3_259
4728_4_7
4732_1_111
4732_2_277
4732_3_111
4732_4_277
4749_1_60
4749_2_27
4749_3_60
4749_4_27
4749_5_60
4749_6_27
4779_1_60
4779_2_1
4779_3_60
4779_4_1
4783_1_111
4783_2_388
4783_3_111
4783_4_388
4789_1_60
4789_2_262
4789_3_60
4789_4_262
4789_5_60
4789_6_262
4814_1_60
4814_2_1
4814_3_60
4814_4_1
4814_5_60
4816_1_60
4816_2_273
4816_3_

6237_2_27
6237_3_60
6237_4_27
6237_5_60
6237_6_27
6242_1_60
6242_2_27
6242_3_60
6242_4_27
6242_5_60
6242_6_27
6246_1_60
6246_2_27
6246_3_60
6246_4_27
6251_1_60
6251_2_296
6251_3_60
6251_4_296
6251_5_60
6418_1_326
6418_2_238
6418_3_326
6418_4_238
6543_1_147
6543_2_243
6543_3_147
6543_4_243
6543_5_147
6543_6_243
6737_1_132
6737_2_234
6737_3_132
6737_4_234
6737_5_132
6742_1_326
6742_2_55
6742_3_326
6742_4_55
6748_1_326
6748_2_329
6748_3_326
6748_4_329
6752_1_326
6752_2_329
6752_3_326
6752_4_329
6761_1_326
6761_2_188
6761_3_326
6761_4_188
6762_1_326
6762_2_234
6762_3_326
6762_4_234
6765_1_326
6765_2_55
6765_3_326
6765_4_55
6774_1_326
6774_2_188
6774_3_326
6774_4_188
6781_1_326
6781_2_20
6781_3_326
6781_4_20
6783_1_326
6783_2_188
6783_3_326
6783_4_188
6786_1_326
6786_2_188
6786_3_326
6786_4_188
6790_1_326
6790_2_20
6790_3_326
6790_4_20
6798_1_326
6798_2_7
6798_3_326
6798_4_7
6892_1_391
6892_2_268
6892_3_391
6892_4_268
6896_1_391
6896_2_238
6896_3_391
6896_4_238
6901_1_391
6901_2_238
6901_3_

8451_4_9
8451_5_60
8453_1_60
8453_2_9
8453_3_60
8453_4_9
8453_5_60
8456_1_60
8456_2_234
8456_3_60
8456_4_234
8456_5_60
8459_1_512
8459_2_274
8459_3_512
8459_4_274
8463_1_509
8463_2_322
8463_3_509
8463_4_322
8463_5_509
8464_1_60
8464_2_160
8464_3_60
8464_4_160
8469_1_60
8469_2_160
8469_3_60
8469_4_160
8469_5_60
8470_1_512
8470_2_274
8470_3_512
8470_4_274
8470_5_512
8472_1_259
8472_2_511
8472_3_259
8472_4_511
8486_1_60
8486_2_40
8486_3_60
8486_4_40
8488_1_509
8488_2_160
8488_3_509
8488_4_160
8490_1_509
8490_2_322
8490_3_509
8490_4_322
8494_1_60
8494_2_40
8494_3_60
8494_4_40
8497_1_509
8497_2_463
8497_3_509
8497_4_463
8497_5_509
8497_6_463
8504_1_60
8504_2_4
8504_3_60
8504_4_4
8504_5_60
8511_1_475
8511_2_9
8511_3_475
8511_4_9
8513_1_512
8513_2_9
8513_3_512
8513_4_9
8522_1_60
8522_2_274
8522_3_60
8522_4_274
8529_1_475
8529_2_261
8529_3_475
8529_4_261
8529_5_475
8529_6_261
8531_1_512
8531_2_9
8531_3_512
8531_4_9
8531_5_512
8532_1_509
8532_2_9
8532_3_509
8532_4_9
8535_1_60
8535_2_9
8535_3_60

IndexError: list index out of range

In [23]:
ed_data

,utterance,label,id
0,"Today, as i was leaving for work in the mornin...",neutral,hit:3_conv:6_1_6
1,Are you fine now?,curiosity,hit:3_conv:6_2_7
2,"Yeah, i'm doing alright now, but with minor in...",approval,hit:3_conv:6_3_6
3,Cool :) Is your car damaged a lot?,curiosity,hit:3_conv:6_4_7
4,"The car was badly damaged, i veered outside th...",neutral,hit:3_conv:6_5_6
...,...,...,...
12073,tha is really cool what was your grade,admiration,hit:12361_conv:24722_4_46
12074,One of my coworkers has been arguing with his ...,neutral,hit:12392_conv:24785_1_791
12075,What are they arguing about?,curiosity,hit:12392_conv:24785_2_829
12076,"Everything and anything. It's annoying, though...",neutral,hit:12392_conv:24785_3_791


In [11]:
ed_idx

0                  hit:3_conv:6_1_6
1                  hit:3_conv:6_2_7
2                  hit:3_conv:6_3_6
3                  hit:3_conv:6_4_7
4                  hit:3_conv:6_5_6
                    ...            
12073     hit:12361_conv:24722_4_46
12074    hit:12392_conv:24785_1_791
12075    hit:12392_conv:24785_2_829
12076    hit:12392_conv:24785_3_791
12077    hit:12392_conv:24785_4_829
Name: id, Length: 12078, dtype: object